In [1]:
import pygame
import sys
import numpy as np
import random
import pandas as pd
import time
import tensorflow as tf
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
BATCH_SIZE=128


pygame 2.1.2 (SDL 2.0.18, Python 3.7.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
width=10
height=10
pixel_size=(50,50)
display_size=(pixel_size[0]*width,pixel_size[1]*height)
record=[]
game_num=1
p=0
step=0

color_dict={
    "wall":(177,177,177),
    "target":(0,177,0),
    "normal":(255,255,255),
    "start":(177,0,0)
}

map_dict={
    "target":(7,2),
    "start":(2,height-1),
    "tank1":(7,4),
    "tank2":(5,2),
    "tank3":(random.randint(3,width-3),random.randint(2,height-3)),

   }

wall_location=[]
for w in range(width):
    wall_location.append((w,0))
    wall_location.append((w,height-1))
    
for h in range(height):    
    wall_location.append((0,h))
    wall_location.append((width-1,h))
    
wall_location.remove(map_dict.get("start"))
wall_location.append((map_dict.get("start")[0],map_dict.get("start")[1]+1))
map_dict['wall']=wall_location

df100=pd.DataFrame(columns=['s','a','s_','r'])
df1=pd.DataFrame(columns=['s','a','s_','r'])
df0=pd.DataFrame(columns=['s','a','s_','r'])

epsilon=0
reward=[-0.02]
gamma=0.9
a_dict={
    0:'up',
    1:'down',
    2:'left',
    3:'right'
}

s_index={}
for i in range(width-2):
    for j in range(height-2):
        s_index.update({(i+1,j+1):(i+(j*(width-2)))})

s_index.update({map_dict.get("start"):(width-2)*(height-2)})   


In [3]:
class tank(pygame.sprite.Sprite):
    def __init__(self,filename,location):
        pygame.sprite.Sprite.__init__(self)
        self.image = pygame.image.load(filename).convert()
        self.image = pygame.transform.scale(self.image,(50,50))

        self.rect = self.image.get_rect()
        self.rect.topleft=(location[0]*pixel_size[0],location[1]*pixel_size[1])
        
        
def draw_map():
    for i in range(width*height):
        w=i-width*(i//width)
        h=i//width
        
        if (w,h)==map_dict.get("target"):
            color=color_dict.get("target")
        elif (w,h)==map_dict.get("start"):
            color=color_dict.get("start")
        elif (w,h) in map_dict.get("wall"):
            color=color_dict.get("wall")
        else:
            color=color_dict.get("normal")
        
        pygame.draw.rect(game_map,color,(w*50,h*50,pixel_size[0],pixel_size[1]))

        
def crash_wall():
    if (person_site[0],person_site[1]) in map_dict.get("wall"):
        return True
    else:
        return False
    
    
def crash_detect(sprite1,sprite2):
    global game_num
    global game_state
    result=pygame.sprite.collide_mask(sprite1,sprite2)
    
    if result:
        if sprite2 in tank1_group:
            print("这是第{}局游戏，我死了".format(game_num))
            if game_num>20:
                print('最后20盘的胜率为{}'.format(record[-20:].count(1)/20))
            record.append(0)
            reward.append(-1)

        elif sprite2 in princess_group:
            print("这是第{}局游戏，我赢了".format(game_num))
            if game_num>20:
                print('最后20盘的胜率为{}'.format(record[-20:].count(1)/20))
            record.append(1)
            reward.append(1)

        game_num+=1
        game_state=1
        
        
def move(direction):
    global person_site
    global step
    if direction=='up':
        person_site[1]-=1
        a= crash_wall()
        if a==True:
            person_site[1]+=1
        
    if direction=='down':
        person_site[1]+=1
        a= crash_wall()
        if a==True:
            person_site[1]-=1
    if direction=='left':
        person_site[0]-=1
        a= crash_wall()
        if a==True:
            person_site[0]+=1
    if direction=='right':
        person_site[0]+=1
        a= crash_wall()
        if a==True:
            person_site[0]-=1
        
    person.rect=(person_site[0]*pixel_size[0],person_site[1]*pixel_size[1],50,50)
    crash_detect(person,tank1)       
    crash_detect(person,tank2)   
    crash_detect(person,tank3)       

    crash_detect(person,princess)         
    step+=1

In [4]:
def pick_a(state):    
    if random.random()>epsilon:
        action=random.randint(0,3)
    else:
        x=model1.predict([state])
        action= x.argmax()
    return action


def one_turn():
    s1=int(person_site[0])
    s2=int(person_site[1])
    a=pick_a([s1,s2])
    move(a_dict.get(a))
    s1_=int(person_site[0])
    s2_=int(person_site[1])
    r=reward[-1]
    trace_collect(s1,s2,a,s1_,s2_,r)
    
    if game_num>50 and step%200==0:
        fit(10)
        
    if game_state==1:           
        reset()

        
def trace_collect(s1,s2,a,s1_,s2_,r):   
    global df100
    global df1
    global df0
    new_trace={
        's1':s1,
        's2':s2,
        'a':a,
        's1_':s1_,
        's2_':s2_,
        'r':r
    }
    if r==1:
        df100=df100.append(new_trace,ignore_index=True)
        df100=df100.iloc[-100:]
        #df100.to_csv('trace100.csv')
    if r==-1:
        df1=df1.append(new_trace,ignore_index=True)
        df1=df1.iloc[-1000:]
        #df1.to_csv('trace1.csv')
    else:
        df0=df0.append(new_trace,ignore_index=True)
        df0=df0.iloc[-4000:]
        #df0.to_csv('trace0.csv')
    
    
def preprocessing(batch_size):
    #df100=pd.read_csv('trace100.csv')
    #df1=pd.read_csv('trace1.csv')
    #df0=pd.read_csv('trace0.csv')
    ds100=df100.sample(batch_size,replace = True)
    ds1=df1.sample(batch_size//2,replace = True)
    ds0=df0.sample(batch_size//2,replace = True)
    train_ds=pd.concat((ds100,ds1,ds0))
    train_ds=train_ds.sample(batch_size)
    return train_ds        

In [5]:
def train_step(train_ds):
    global p
    s = np.int32(np.stack((train_ds['s1'],train_ds['s2']),1))
    a = np.array(train_ds['a'])
    s_ = np.int32(np.stack((train_ds['s1_'],train_ds['s2_']),1))
    r = np.array(train_ds['r'])
    q_=model1(s_)
    a_=np.argmax(q_,axis=1)

    
    c1=np.zeros((len(train_ds),4))   #A方向外全为0的矩阵
    for i in range(len(a)):
        c1[i][int(a[i])]=1
    
    c2=np.ones((len(train_ds),4))    #A方向外全为1的矩阵
    for i in range(len(a)):
        c2[i][int(a[i])]=0
        
    c3=np.zeros((len(train_ds),4))   #A_方向外全为0的矩阵
    for i in range(len(a_)):
        c3[i][int(a_[i])]=1
        
    c4=np.ones((len(train_ds),4))   #A_方向外全为1的矩阵
    for i in range(len(a_)):
        c4[i][int(a_[i])]=0
    
    d=np.ones(len(r))
    for i in range(len(r)):
        if r[i]==1:
            d[i]=0
        elif r[i]==-1:
            d[i]=0
        else:
            pass
        
    qloss=model1(s)
    qt=model2(s_)   
    #qtarget=tf.reduce_max(qt,axis=1)*gamma*d+r
    qt=tf.reduce_max(qt*c3,axis=1)
    qt=qt*gamma*d+r
    qtloss=tf.transpose([qt,qt,qt,qt])*c1+qloss*c2
       
    with tf.GradientTape(persistent=True) as tape:
        #loss函数
        q=model1(s)        
        loss=tf.reduce_sum(tf.square(qtloss-q))
        p=float(loss)
    
    gradient=tape.gradient(loss,model1.trainable_variables)
    opt.apply_gradients(zip(gradient,model1.trainable_variables))
    del tape    

In [6]:
model1=tf.keras.Sequential([
    tf.keras.layers.Dense(128,input_shape=(BATCH_SIZE,2),activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(4)       #a,qmax=r+gamma*q_max
   ]
)

model2=tf.keras.Sequential([
    tf.keras.layers.Dense(128,input_shape=(BATCH_SIZE,2),activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(4)       #a,qmax=r+gamma*q_max
   ]
)

opt=tf.keras.optimizers.Adam(learning_rate=0.001)

In [7]:
def fit(epochs):
    for epoch in range(epochs):
        train_ds=preprocessing(BATCH_SIZE)
        for i in range(50):
            train_step(train_ds)
        print(str(p)+'.',end='')
        
        if (epoch+1)%10==0:
            print("EPOCH:",epoch+1)

    
def load_weight():
    model1.save_weights('DQN.h5')
    model2.load_weights('DQN.h5')
    
    
def reset():
    global person_site
    global game_state
    person_site=[map_dict.get("start")[0],map_dict.get("start")[1]]
    person.rect=(person_site[0]*pixel_size[0],person_site[1]*pixel_size[1],50,50)
    game_state=0
    reward.append(-0.02)
    
    if game_num>70 and game_num%20==0:    
        load_weight()
        #debug()
        

def debug():
    for i in range(width-2):
        for j in range(height-2):
            result=model1.predict([[i+1,j+1]])
            print(i+1,j+1)
            print(result)

In [8]:
pygame.init()
clock = pygame.time.Clock()
pygame.display.set_caption("AI训练游戏")

screen = pygame.display.set_mode(display_size)
game_map=pygame.surface.Surface(display_size)
draw_map()

tank1_group = pygame.sprite.Group()
tank2_group = pygame.sprite.Group()
princess_group = pygame.sprite.Group()
person_group=pygame.sprite.Group()

tank1 = tank("img/tank.jpg",map_dict.get("tank1"))
tank2 = tank("img/tank.jpg",map_dict.get("tank2"))
tank3 = tank("img/tank.jpg",map_dict.get("tank3"))

tank1_group.add(tank1)
tank1_group.add(tank2)
tank1_group.add(tank3)


princess=tank('img/sidalin.jpeg',map_dict.get("target"))
princess_group.add(princess)

person_site=[map_dict.get("start")[0],map_dict.get("start")[1]]
person=tank('img/maintank.jpeg',person_site)
person_group.add(person)

reset()

while True:
    clock.tick(60)

    screen.blit(game_map,(0,0))
    tank1_group.draw(screen)
    princess_group.draw(screen)
    screen.blit(person.image,person.rect)
    tank2_group.draw(screen)
    
    
    one_turn()



    if game_num==50:
        epsilon=0.2
    elif game_num==100:
        epsilon=0.3   
    elif game_num==150:
        epsilon=0.4
    elif game_num==200:
        epsilon=0.5
    elif game_num==250:
        epsilon=0.6
    elif game_num==275:
        epsilon=0.7
    elif game_num==300:
        epsilon=0.8
    elif game_num==325:
        epsilon=0.9
    elif game_num==350:
        epsilon=0.95
    elif game_num==375:
        epsilon=0.99
        
    for event in pygame.event.get():
        if event.type ==pygame.QUIT:
            pygame.quit()
            sys.exit()
            

        
#       if event.type ==pygame.KEYDOWN:
#           if event.key == pygame.K_UP:
#               move('up')
#           if event.key == pygame.K_DOWN:
#               move('down')
#           if event.key == pygame.K_LEFT:
#               move('left')
#           if event.key == pygame.K_RIGHT:
#               move('right')
#           if event.key == pygame.K_SPACE:
                
                
     
    pygame.display.update()

这是第1局游戏，我死了
这是第2局游戏，我死了
这是第3局游戏，我赢了
这是第4局游戏，我死了
这是第5局游戏，我死了
这是第6局游戏，我死了
这是第7局游戏，我死了
这是第8局游戏，我死了
这是第9局游戏，我死了
这是第10局游戏，我赢了
这是第11局游戏，我死了
这是第12局游戏，我死了
这是第13局游戏，我死了
这是第14局游戏，我死了
这是第15局游戏，我死了
这是第16局游戏，我死了
这是第17局游戏，我死了
这是第18局游戏，我死了
这是第19局游戏，我死了
这是第20局游戏，我死了
这是第21局游戏，我死了
最后20盘的胜率为0.1
这是第22局游戏，我死了
最后20盘的胜率为0.1
这是第23局游戏，我死了
最后20盘的胜率为0.1
这是第24局游戏，我死了
最后20盘的胜率为0.05
这是第25局游戏，我死了
最后20盘的胜率为0.05
这是第26局游戏，我死了
最后20盘的胜率为0.05
这是第27局游戏，我死了
最后20盘的胜率为0.05
这是第28局游戏，我死了
最后20盘的胜率为0.05
这是第29局游戏，我死了
最后20盘的胜率为0.05
这是第30局游戏，我死了
最后20盘的胜率为0.05
这是第31局游戏，我死了
最后20盘的胜率为0.0
这是第32局游戏，我死了
最后20盘的胜率为0.0
这是第33局游戏，我死了
最后20盘的胜率为0.0
这是第34局游戏，我死了
最后20盘的胜率为0.0
这是第35局游戏，我死了
最后20盘的胜率为0.0
这是第36局游戏，我死了
最后20盘的胜率为0.0
这是第37局游戏，我死了
最后20盘的胜率为0.0
这是第38局游戏，我死了
最后20盘的胜率为0.0
这是第39局游戏，我死了
最后20盘的胜率为0.0
这是第40局游戏，我死了
最后20盘的胜率为0.0
这是第41局游戏，我死了
最后20盘的胜率为0.0
这是第42局游戏，我死了
最后20盘的胜率为0.0
这是第43局游戏，我死了
最后20盘的胜率为0.0
这是第44局游戏，我赢了
最后20盘的胜率为0.0
这是第45局游戏，我死了
最后20盘的胜率为0.05
这是第46局游戏，我死了
最后20盘的胜率为0.05
这是第47局游戏，我死了
最后20盘的胜率为0.05
这是第48局游戏，我死了
最后20盘的胜率为0.05
这是第49局游戏，我

0.003639897098764777.0.000600811094045639.0.008494224399328232.0.0024404358118772507.0.0008112433133646846.0.005342656280845404.0.013288811780512333.0.0014293678104877472.0.010599767789244652.0.00243547186255455.EPOCH: 10
这是第119局游戏，我死了
最后20盘的胜率为0.1
这是第120局游戏，我死了
最后20盘的胜率为0.1
这是第121局游戏，我死了
最后20盘的胜率为0.1
这是第122局游戏，我死了
最后20盘的胜率为0.1
0.08906055241823196.0.017245152965188026.0.007675850298255682.0.019198626279830933.0.022134561091661453.0.006774735637009144.0.007733937352895737.0.012161028571426868.0.0022567552514374256.0.009211225435137749.EPOCH: 10
这是第123局游戏，我死了
最后20盘的胜率为0.05
0.002599364845082164.0.004751242697238922.0.002450125990435481.0.003453400218859315.0.0010421881452202797.0.0003257333883084357.0.03277134895324707.0.0025068135000765324.0.013006621971726418.0.0022822602186352015.EPOCH: 10
这是第124局游戏，我赢了
最后20盘的胜率为0.05
这是第125局游戏，我死了
最后20盘的胜率为0.1
这是第126局游戏，我死了
最后20盘的胜率为0.1
这是第127局游戏，我死了
最后20盘的胜率为0.1
这是第128局游戏，我死了
最后20盘的胜率为0.1
这是第129局游戏，我死了
最后20盘的胜率为0.1
这是第130局游戏，我死了
最后20盘的胜率为0.1
这是第131局游戏

0.0063992636278271675.0.000522158807143569.0.0003616985632106662.0.0004881137574557215.0.07353115826845169.0.000857699429616332.0.001132405479438603.0.0008403176907449961.0.00030533724930137396.0.0002196273417212069.EPOCH: 10
这是第209局游戏，我死了
最后20盘的胜率为0.2
0.0006980509497225285.0.0009319673990830779.0.00036193677806295455.0.0006152821006253362.0.0003256137715652585.0.0028636991046369076.0.002422960475087166.0.00288137374445796.0.003950794227421284.0.00012738531222566962.EPOCH: 10
这是第210局游戏，我赢了
最后20盘的胜率为0.2
这是第211局游戏，我赢了
最后20盘的胜率为0.25
这是第212局游戏，我赢了
最后20盘的胜率为0.3
0.00047542815445922315.0.0013631004840135574.0.07236183434724808.0.0013720572460442781.0.002248842502012849.0.004717400297522545.0.004151298198848963.0.0011487301671877503.0.0245151836425066.0.005401673261076212.EPOCH: 10
这是第213局游戏，我赢了
最后20盘的胜率为0.35
这是第214局游戏，我死了
最后20盘的胜率为0.35
0.002920635510236025.0.023515362292528152.0.0008094317745417356.0.0008062588167376816.0.07225629687309265.0.0011858583893626928.0.0008081143023446202.0.0024381

0.03878061845898628.0.09248083829879761.0.15493851900100708.0.0020917414221912622.0.16674529016017914.0.009706911630928516.0.062103185802698135.0.01862609013915062.0.15554486215114594.0.007649358361959457.EPOCH: 10
这是第290局游戏，我赢了
最后20盘的胜率为0.3
这是第291局游戏，我赢了
最后20盘的胜率为0.35
这是第292局游戏，我死了
最后20盘的胜率为0.4
这是第293局游戏，我赢了
最后20盘的胜率为0.4
0.07704685628414154.0.031893160194158554.0.00497762206941843.0.04155220091342926.0.04012930393218994.0.22848094999790192.0.022164322435855865.0.07611741125583649.0.11703050881624222.0.001433463068678975.EPOCH: 10
这是第294局游戏，我死了
最后20盘的胜率为0.45
0.00048517249524593353.0.0023781978525221348.0.05649646371603012.0.005045190919190645.0.009183048270642757.0.04004252701997757.0.005212554708123207.0.00914138462394476.0.03683686628937721.0.0004609720199368894.EPOCH: 10
0.1764666587114334.0.025709642097353935.0.01247416716068983.0.002183077624067664.0.01626097597181797.0.00887907762080431.0.0312751829624176.0.043192554265260696.0.0003963919298257679.0.0071372766979038715.EPOCH: 10


0.0044184192083776.0.0011771206045523286.0.006356049329042435.0.0010133214527741075.0.08083383738994598.0.012764880433678627.0.004868711810559034.0.00863325409591198.0.02345113642513752.0.03609341010451317.EPOCH: 10
这是第346局游戏，我赢了
最后20盘的胜率为0.8
这是第347局游戏，我赢了
最后20盘的胜率为0.8
0.003465496003627777.0.0037527179811149836.0.007194558158516884.0.001517134951427579.0.010947616770863533.0.012771708890795708.0.0003450771910138428.0.001338110538199544.0.05040734261274338.0.02235473319888115.EPOCH: 10
0.024458173662424088.0.2993859052658081.0.023092450574040413.0.013941234908998013.0.0024063922464847565.0.004878988489508629.0.000626348948571831.0.00045897249947302043.0.003984970506280661.0.0064518447034060955.EPOCH: 10
这是第348局游戏，我赢了
最后20盘的胜率为0.85
这是第349局游戏，我赢了
最后20盘的胜率为0.85
这是第350局游戏，我赢了
最后20盘的胜率为0.85
这是第351局游戏，我赢了
最后20盘的胜率为0.85
这是第352局游戏，我赢了
最后20盘的胜率为0.85
这是第353局游戏，我赢了
最后20盘的胜率为0.85
这是第354局游戏，我死了
最后20盘的胜率为0.85
这是第355局游戏，我死了
最后20盘的胜率为0.85
这是第356局游戏，我赢了
最后20盘的胜率为0.8
这是第357局游戏，我赢了
最后20盘的胜率为0.8
这是第358局游戏，

SystemExit: 

d:\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


Double Dqn的训练相比DQN要稳定许多
但是还是存在一定的过估计问题
在最好策略形成以后，如果不停止训练，会退化。很容易在一些地方左右横跳